In [1]:
import torch
import pandas as pd
import numpy as np
import pickle
import torchvision.models as models

In [2]:
dim = 50

In [ ]:
def exShapeMatrix(m_path='ShapeNet.pth', d=300):
    m_path = m_path.replace('Net', 'Net_'+str(d)+'d')
    pretrained_dict = torch.load(m_path)
    M = pretrained_dict['model']['classifier.0.weight']
    # resave
    torch.save({'shape2vec': M}, f'shape2vec.{d}d')
    print("Saved!")
# M = pretrained_dict['model']['classifier.0.weight'].cpu().detach().numpy()

exShapeMatrix(d=dim)

In [3]:
def get_label_dict():
    f = open('./chinese_labels', 'rb')
    label_dict = pickle.load(f)
    f.close()
    return label_dict

id2char = get_label_dict()
char2id = {x: y for x, y in zip(id2char.values(), id2char.keys())}

def getAllReady():
    id2char = get_label_dict()
    char2id = {x: y for x, y in zip(id2char.values(), id2char.keys())}
    M = torch.load(f'./shape2vec.{dim}d')['shape2vec']
    print(M.shape)
    return M
    
M = getAllReady()

torch.Size([3755, 50])


In [4]:
def getTopKSim(q_char, K=20):
    id = char2id[q_char]
    sim_lst = []
    for i in range(M.shape[0]):
        sim = torch.cosine_similarity(M[id], M[i], dim=-1)
        sim_lst.append((sim, i))
    sim_lst.sort(key=lambda x: x[0], reverse=True)
    topK_id = sim_lst[1:K+1]
    topK_char = []
    for _, id in topK_id:
        topK_char.append(id2char[id])
    return topK_char


#### Test

In [ ]:
# 50d
print("鹿：", getTopKSim("鹿", 20))
print("于：", getTopKSim("于", 20))
print("茵：", getTopKSim("茵", 20))
print("少：", getTopKSim("少", 20))
print("饮：", getTopKSim("饮", 20))

In [ ]:
# 100d
print("鹿：", getTopKSim("鹿", 20))
print("于：", getTopKSim("于", 20))
print("茵：", getTopKSim("茵", 20))
print("少：", getTopKSim("少", 20))
print("饮：", getTopKSim("饮", 20))

In [ ]:
# 300d
print("鹿：", getTopKSim("鹿", 20))
print("于：", getTopKSim("于", 20))
print("茵：", getTopKSim("茵", 20))
print("少：", getTopKSim("少", 20))
print("饮：", getTopKSim("饮", 20))

#### 形似字字典

In [ ]:
def getTopKSimDict(K=20):
    sim_dct = {}
    for q_char in char2id.keys():
        id = char2id[q_char]
        sim_lst = []
        for i in range(M.shape[0]):
            sim = torch.cosine_similarity(M[id], M[i], dim=-1)
            sim_lst.append((sim, i))
        sim_lst.sort(key=lambda x: x[0], reverse=True)
        topK_id = sim_lst[1:K+1]
        topK_char = []
        for _, id in topK_id:
            topK_char.append(id2char[id])
        sim_dct[q_char] = topK_char
    return sim_dct

sim_dct = getTopKSimDict()
torch.save(sim_dct, "./sim_dct")

In [ ]:
def get_sim_dict():
    f = open('./sim_dct', 'rb')
    sim_dict = pickle.load(f)
    f.close()
    return sim_dict

sim_dct = get_sim_dict()

In [ ]:
sim_dct = torch.load('sim_dct')

In [ ]:
type(sim_dct)

In [8]:
torch.cosine_similarity(M[char2id["于"]], M[char2id["干"]], dim=-1)

tensor(0.5860, device='cuda:0')